## Import the lib

In [1]:
import os
import sys
import math
import torch
import argparse
import torch.nn as nn
from torchview import draw_graph

## Residual Block

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels: int = 256):
        super(ResidualBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = in_channels

        self.reflectionpad2d = 1
        self.kernel_size = 3
        self.stride_size = 1

        self.layers = []

        for index in range(2):
            self.layers.append(nn.ReflectionPad2d(self.reflectionpad2d))

            self.layers.append(
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=self.out_channels,
                    kernel_size=self.kernel_size,
                    stride=self.stride_size,
                )
            )

            self.layers.append(nn.InstanceNorm2d(num_features=self.out_channels))

            if index != 1:
                self.layers.append(nn.ReLU())

        self.residualBlock = nn.Sequential(*self.layers)

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            return x + self.residualBlock(x)
        else:
            raise ValueError("Input should be the tensor type".capitalize())


if __name__ == "__main__":
    image_channels = 256
    residual = nn.Sequential(
        *[ResidualBlock(in_channels=image_channels) for _ in range(3)]
    )

    print(residual)

## Encoder 

In [ ]:
class Encoder(nn.Module):

    def __init__(self, in_channels: int = 3, sharedBlocks=None):
        super(Encoder, self).__init__()

        self.in_channels = in_channels
        self.out_channels = int(math.pow(2, self.in_channels + self.in_channels))
        self.kerenl_size = (self.in_channels * 2) + 1

        if not isinstance(sharedBlocks, ResidualBlock):
            raise ValueError(
                "shared_block must be an instance of ResidualBlock".capitalize()
            )

        self.sharedBlocks = sharedBlocks

        self.modelBlocks = list()
        self.downLayers = list()

        self.modelBlocks.append(
            nn.Sequential(
                nn.ReflectionPad2d(padding=self.in_channels),
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=self.out_channels,
                    kernel_size=self.kerenl_size,
                ),
                nn.InstanceNorm2d(num_features=self.out_channels),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
            )
        )

        for _ in range(2):
            self.downLayers.append(
                nn.Conv2d(
                    in_channels=self.out_channels,
                    out_channels=self.out_channels * 2,
                    kernel_size=self.kerenl_size // 2,
                    stride=(self.in_channels // self.in_channels) + 1,
                    padding=self.in_channels // self.in_channels,
                )
            )
            self.downLayers.append(
                nn.InstanceNorm2d(num_features=self.out_channels * 2)
            )
            self.downLayers.append(nn.ReLU())

            self.out_channels *= 2

        self.modelBlocks.append(nn.Sequential(*self.downLayers))

        self.modelBlocks.append(
            nn.Sequential(
                *[ResidualBlock(in_channels=self.out_channels) for _ in range(3)]
            )
        )

        self.modelBlocks = nn.Sequential(*self.modelBlocks)

    def reparameterization(self, mu: torch.Tensor):
        if isinstance(mu, torch.Tensor):
            z = torch.randn_like(mu)
            return mu + z
        else:
            raise ValueError("Input should be the tensor type".capitalize())

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.modelBlocks(x)
            mu = self.sharedBlocks(x)
            z = self.reparameterization(mu)

            return mu, z
        else:
            raise ValueError("Input should be the tensor type".capitalize())


if __name__ == "__main__":
    in_channels = 3

    batch_size = 1
    image_size = 128

    shared_E = ResidualBlock(
        in_channels=int(in_channels * (math.pow(2, 8) - 1) / in_channels + 1)
    )

    encoder1 = Encoder(in_channels=in_channels, sharedBlocks=shared_E)
    encoder2 = Encoder(in_channels=in_channels, sharedBlocks=shared_E)

    mu1, z1 = encoder1(torch.randn(batch_size, in_channels, image_size, image_size))
    mu2, z2 = encoder2(torch.randn(batch_size, in_channels, image_size, image_size))

    assert (
        mu1.size() == mu2.size() == z1.size() == z2.size()
    ), "Shape mismatch(mu1, mu2) and (z1, z2)".capitalize()

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels: int = 256, sharedBlock: ResidualBlock = None):
        super(Generator, self).__init__()

        self.in_channels = in_channels
        self.out_channels = self.in_channels

        self.kernel_size = int(math.sqrt(math.sqrt(self.in_channels)))
        self.stride_size = int(math.sqrt(self.kernel_size))
        self.padding_size = self.stride_size // self.stride_size

        if isinstance(sharedBlock, ResidualBlock):
            self.sharedBlock = sharedBlock
        else:
            raise ValueError(
                "shared_block must be an instance of ResidualBlock".capitalize()
            )

        self.modelBlocks = []
        self.upsampleBlocks = []

        self.modelBlocks.append(
            nn.Sequential(
                *[ResidualBlock(in_channels=self.in_channels) for _ in range(3)]
            )
        )

        for _ in range(2):
            self.upsampleBlocks.append(
                nn.ConvTranspose2d(
                    in_channels=self.in_channels,
                    out_channels=self.in_channels // 2,
                    kernel_size=self.kernel_size,
                    stride=self.stride_size,
                    padding=self.padding_size,
                )
            )

            self.upsampleBlocks.append(
                nn.InstanceNorm2d(num_features=self.in_channels // 2)
            )
            self.upsampleBlocks.append(nn.LeakyReLU(negative_slope=0.2, inplace=True))

            self.in_channels //= 2

        self.modelBlocks.append(nn.Sequential(*self.upsampleBlocks))

        self.modelBlocks.append(
            nn.Sequential(
                nn.ReflectionPad2d(padding=self.kernel_size - 1),
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=self.kernel_size - 1,
                    kernel_size=self.kernel_size + 3,
                ),
                nn.Tanh(),
            )
        )

        self.modelBlocks = nn.Sequential(*self.modelBlocks)

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.sharedBlock(x)
            return self.modelBlocks(x)
        else:
            raise ValueError("Input should be the tensor type".capitalize())


if __name__ == "__main__":
    image_channels = 256

    batch_size = 1
    image_size = 32

    shared_G = ResidualBlock(in_channels=image_channels)

    netG1 = Generator(
        in_channels=image_channels,
        sharedBlock=ResidualBlock(in_channels=image_channels),
    )
    netG2 = Generator(
        in_channels=image_channels,
        sharedBlock=ResidualBlock(in_channels=image_channels),
    )

    generatedImage1 = netG1(
        torch.randn(batch_size, image_channels, image_size, image_size)
    )
    generatedImage2 = netG2(
        torch.randn(batch_size, image_channels, image_size, image_size)
    )

    assert (
        generatedImage1.size() == generatedImage2.size()
    ), "Shape mismatch(generatedImage1, generatedImage2)".capitalize()